# Add Unconfigured Ports to VLAN Default

In [ ]:
import exsh
import re

try:
    # Capture output of "show ports vlan"
    vlan_output = exsh.clicmd("show ports vlan", capture=True)

    # Identify completely unconfigured ports (those with "None None")
    unassigned_ports = []

    # Process each line in the VLAN output
    for line in vlan_output.splitlines():
        # Match lines with the pattern "X/Y None None" (completely unconfigured ports)
        match = re.match(r"^\s*(\d+):(\d+)\s+None\s+None", line)
        if match:
            port_group = int(match.group(1))  # First part of the port (e.g., 1 in 1:1)
            port_number = int(match.group(2))  # Second part of the port (e.g., 1 in 1:1)
            port = f"{port_group}:{port_number}"

            # Exclude ports numbered 49 and above
            if port_number < 49:
                unassigned_ports.append((port_group, port_number, port))

    # Sort ports numerically by port group and port number
    unassigned_ports.sort(key=lambda x: (x[0], x[1]))

    # If there are unassigned ports, add them to the default VLAN
    if unassigned_ports:
        for port in unassigned_ports:
            port_str = port[2]  # Get the port string (e.g., "1:1")
            # Construct the command to add the port to the default VLAN
            cmd = f"configure vlan Default add ports {port_str} untagged"
            # Execute the command
            exsh.clicmd(cmd)
            print(f"[+] Successfully added port {port_str} to VLAN Default")
    else:
        print("[*] No unconfigured ports found.")

except Exception as e:
    # Handle any exceptions that occur
    print(f"[-] Error: {e}")